# [Fraud Detection](https://www.kaggle.com/ntnu-testimon/paysim1/downloads/paysim1.zip/2) demo

## 1. Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import grpc
import hydro_serving_grpc as hs
from grpc import ssl_channel_credentials
from sklearn import preprocessing

## 2. Reading data

In [2]:
df = pd.read_csv("../data/PS_20174392719_1491204439457_log.csv")

CPU times: user 12.3 s, sys: 2.3 s, total: 14.6 s
Wall time: 14 s


In [3]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


### Encoding transaction type category labels

In [4]:
le = preprocessing.LabelEncoder()
df.type = le.fit_transform(df.type)

In [5]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,3,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,3,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,4,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,1,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,3,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [6]:
transactions, labels = df.drop(columns=['nameOrig', 'nameDest', 'isFraud', 'isFlaggedFraud'], axis=1), df['isFraud']

In [9]:
transactions.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,1,3,9839.64,170136.0,160296.36,0.0,0.0
1,1,3,1864.28,21249.0,19384.72,0.0,0.0
2,1,4,181.00,181.0,0.00,0.0,0.0
3,1,1,181.00,181.0,0.00,21182.0,0.0
4,1,3,11668.14,41554.0,29885.86,0.0,0.0


In [16]:
print(f'Transaction parameters used for fraud detection: \n {list(transactions.columns)}')

Transaction parameters used for fraud detection: 
 ['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']


In [27]:
example_transaction, is_fraud = transactions.iloc[0], labels[0]

In [37]:
example_transaction = np.array(example_transaction)
example_transaction = example_transaction.reshape((1, 7))

## 3. Connecting to server

In [20]:
channel = grpc.secure_channel("dev.k8s.hydrosphere.io", credentials=ssl_channel_credentials())
stub = hs.PredictionServiceStub(channel)

In [21]:
model_spec = hs.ModelSpec(name="fraud_rf", signature_name="infer")

## 4. Form tensor

In [39]:
tensor_shape = hs.TensorShapeProto(
        dim=[hs.TensorShapeProto.Dim(size=item) for item in example_transaction.shape])
transaction_tensor = hs.TensorProto(dtype=hs.DT_DOUBLE,
                                    double_val=example_transaction.flatten(),
                                    tensor_shape=tensor_shape)

## 5. Send request to model

In [40]:
request = hs.PredictRequest(model_spec=model_spec, inputs={"features":transaction_tensor})
result = stub.Predict(request)
result_tensor = result.outputs.get('is_fraud')

In [42]:
print(f'True answer: {is_fraud}')
print(f'Predicted answer: {result_tensor.bool_val[0]}')

True answer: 0
Predicted answer: False
